# Mobile price classification with multilayer perceptron
## PyTorch
Erik Matovič and Jakub Horvat

[Dataset](https://www.kaggle.com/datasets/iabhishekofficial/mobile-price-classification?select=train.csv)

### 0. Imports

In [25]:
import wandb
from pprint import pprint
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
from utils import split_train_val
from typing import Tuple, List
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report, accuracy_score

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
print(use_cuda, device)

True cuda:0


In [26]:
wandb.login()

True

### 2. Data transformation
 - no null values
 - load train and test datasets & from test dataset make validation and test dataset

In [27]:
# read datasets
df_train = pd.read_csv('../data/train.csv', sep=',')
df_test = df_train[-200:]
df_train = df_train[:-200]
df_train

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,527,0,0.7,1,4,0,53,0.9,128,1,...,67,1924,1302,13,7,13,0,0,0,0
1796,793,1,2.8,1,1,1,64,0.4,179,4,...,818,1006,2296,12,0,5,1,1,0,1
1797,873,1,0.5,1,6,0,8,0.6,152,7,...,1257,1411,575,15,8,7,1,0,1,0
1798,722,0,1.1,0,12,0,12,1.0,93,6,...,138,1371,2359,13,6,3,0,0,0,1


In [28]:
df_test

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
1800,1358,0,1.7,0,0,0,60,0.6,147,3,...,889,1635,440,14,10,13,1,0,0,0
1801,1035,1,1.9,1,8,0,11,1.0,173,6,...,831,1709,2002,17,8,5,1,1,0,1
1802,1289,0,0.8,1,3,1,16,0.9,161,6,...,713,1854,3167,14,8,20,1,1,0,3
1803,1445,0,2.2,1,2,0,37,0.4,82,2,...,527,1393,3480,17,16,17,1,1,1,3
1804,1702,1,2.3,1,12,1,47,0.6,107,2,...,1284,1643,595,10,0,19,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


In [29]:
X_train = df_train.values[:, :-1]       # store the inputs
X_train = X_train.astype('float32')     # input data as floats
X_train

array([[8.420e+02, 0.000e+00, 2.200e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [1.021e+03, 1.000e+00, 5.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       [5.630e+02, 1.000e+00, 5.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [8.730e+02, 1.000e+00, 5.000e-01, ..., 1.000e+00, 0.000e+00,
        1.000e+00],
       [7.220e+02, 0.000e+00, 1.100e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.250e+03, 1.000e+00, 1.500e+00, ..., 1.000e+00, 1.000e+00,
        1.000e+00]], dtype=float32)

In [30]:
X_test = df_test.values[:, :-1]       # store the inputs
X_test = X_test.astype('float32')     # input data as floats
X_test

array([[1.358e+03, 0.000e+00, 1.700e+00, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [1.035e+03, 1.000e+00, 1.900e+00, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       [1.289e+03, 0.000e+00, 8.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [1.911e+03, 0.000e+00, 9.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       [1.512e+03, 0.000e+00, 9.000e-01, ..., 1.000e+00, 1.000e+00,
        1.000e+00],
       [5.100e+02, 1.000e+00, 2.000e+00, ..., 1.000e+00, 1.000e+00,
        1.000e+00]], dtype=float32)

In [31]:
y_train = df_train.values[:, -1]  # store the outputs
y_train = y_train.astype('int')     # input data as floats
y_train

array([1, 2, 2, ..., 0, 1, 2])

In [32]:
y_test = df_test.values[:, -1]  # store the outputs
y_test = y_test.astype('int')     # input data as floats
y_test

array([0, 1, 3, 3, 1, 2, 1, 0, 0, 1, 0, 1, 0, 0, 1, 3, 3, 1, 0, 2, 0, 3,
       1, 2, 2, 1, 2, 3, 3, 0, 0, 1, 2, 2, 2, 1, 2, 1, 0, 2, 3, 0, 0, 3,
       0, 3, 0, 0, 3, 3, 0, 3, 0, 0, 0, 0, 3, 3, 2, 0, 0, 0, 0, 3, 0, 2,
       3, 2, 3, 3, 2, 3, 1, 0, 2, 0, 0, 1, 1, 1, 2, 2, 0, 0, 3, 0, 1, 2,
       0, 3, 1, 0, 0, 3, 0, 1, 0, 3, 3, 0, 1, 2, 2, 2, 3, 0, 1, 3, 3, 1,
       3, 0, 1, 0, 0, 2, 1, 0, 2, 0, 3, 1, 0, 2, 1, 2, 1, 3, 0, 2, 3, 1,
       1, 1, 2, 1, 3, 2, 2, 2, 2, 3, 1, 0, 0, 0, 0, 2, 1, 2, 1, 0, 1, 3,
       0, 3, 0, 3, 3, 1, 2, 0, 2, 1, 1, 0, 2, 1, 0, 2, 2, 1, 0, 3, 1, 3,
       0, 2, 1, 1, 1, 1, 2, 2, 1, 3, 1, 0, 1, 1, 0, 3, 0, 3, 3, 0, 2, 3,
       0, 3])

In [33]:
# make dataset from X and y
train_ds = TensorDataset(torch.tensor(X_train), torch.tensor(y_train, dtype=torch.long))
train_ds

In [34]:
# make dataset from X and y
test_ds = TensorDataset(torch.tensor(X_test), torch.tensor(y_test, dtype=torch.long))
test_ds

In [35]:
default_config = {
    'dense1': 256,
    'learning_rate': 0.01,
    'epochs': 100,
    'batch_size': 128
}

In [36]:
tf_train, tf_val = split_train_val(train_ds)

print(tf_train.dataset)
print(tf_val.dataset)

## Model

In [39]:
class MLP(nn.Module):
    """ 
    Model class.
    :param Module from torch.nn
    """
    def __init__(self, n_inputs: int, n_classes: int, lr: float, config) -> None:
        """
        Model elements init.
        """
        super(MLP, self).__init__()
        self.n_inputs = n_inputs
        self.n_classes = n_classes
        self.lr = lr
        self.hidden_size = config.dense1
        # self.hidden_size = hidden_size
    
        # input to first hidden layer
        # pridať hlbku aj šírku siete
        # skúsiť normalizovať dáta
        self.dense1 = nn.Linear(n_inputs, self.hidden_size)
        self.dense2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.dense3 = nn.Linear(self.hidden_size, self.hidden_size)
        self.dense4 = nn.Linear(self.hidden_size, self.hidden_size)
        self.dense5 = nn.Linear(self.hidden_size, self.hidden_size)
        self.dense6 = nn.Linear(self.hidden_size, self.n_classes)
        
        # activation f
        self.relu = nn.ReLU()
        # prv začat bez dropout, ptm pridat, pozrieť sa na parametre 
        # dropout pomaha proti preuceniu
        self.droput = nn.Dropout(p=0.05)
 
    def forward(self, X:torch.Tensor) -> torch.Tensor:
        """
        Feed forward
        """
        # print(type(X))
        #print(self.n_inputs)
        # input to first hidden layer
        output = self.dense1(X)
        output = self.relu(output)
        output = self.droput(output)
        
        # second hidden layer
        output = self.dense2(output)
        output = self.relu(output)
        output = self.droput(output)

        # third hidden layer
        output = self.dense3(output)
        output = self.relu(output)
        output = self.droput(output)
        
        # second hidden layer
        output = self.dense4(output)
        output = self.relu(output)
        output = self.droput(output)
        
        # second hidden layer
        output = self.dense5(output)
        output = self.relu(output)
        output = self.droput(output)

        # final layer and output
        output = self.dense6(output)
        # print(type(output))

        # print(type(output))
        return output

In [42]:
sweep_config = {
    'name': 'NSIETE_zad1_final',
    'method': 'bayes',
    'metric': {
        'name': 'validation_accuracy', 
        'goal': 'maximize' 
    },
    'early_terminate':{
        'type': 'hyperband',
        'min_iter': 5
    },
    'parameters': {
        'batch_size': {
            'values': [128, 256, 512, 1024]
        },
        'epochs': {
            'values': [100, 500, 1000]
        },
        'learning_rate': {
            'min': 0.001,
            'max': 0.1
        },
        'dense1': {
            'values': [256, 512]
        },
    }
}

pprint(sweep_config)

sweep_id = wandb.sweep(sweep_config)

{'early_terminate': {'min_iter': 5, 'type': 'hyperband'},
 'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'validation_accuracy'},
 'name': 'NSIETE_zad1_final',
 'parameters': {'batch_size': {'values': [128, 256, 512, 1024]},
                'dense1': {'values': [256, 512]},
                'epochs': {'values': [100, 500, 1000]},
                'learning_rate': {'max': 0.1, 'min': 0.001}}}
Create sweep with ID: 1stel95e
Sweep URL: https://wandb.ai/xmatovice/uncategorized/sweeps/1stel95e


In [43]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    return correct


def train_mlp(n_epochs, mlp, optimizer, loss_fn, 
              train_dl, val_dl, device, batch_size):
    # init train lists for statistics
    loss_train, accuracy_train = list(), list()

    # init validation lists for statistics
    loss_validation, accuracy_validation = list(), list()

    # enumerate epochs
    for epoch in range(n_epochs):
        # init epoch train counters
        epoch_train_accuracy, epoch_train_total, epoch_train_true, epoch_train_loss = 0, 0, 0, 0

        # init epoch validation counters
        epoch_validation_accuracy, epoch_validation_total, \
            epoch_validation_true, epoch_validation_loss = 0, 0, 0, 0

        # enumerate mini batches
        for idx, (X_batch, y_batch) in enumerate(train_dl):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            # Make prediction logits with model
            y_logits = mlp(X_batch)
            # Perform softmax calculation on logits across dimension 1 to get prediction probabilities
            y_pred_probs = torch.softmax(y_logits, dim=1) 
            # go from logits -> prediction probabilities -> prediction labels
            y_pred = torch.argmax(y_pred_probs, dim=1) 
            
            loss = loss_fn(y_logits, y_batch)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

            # update train counters
            epoch_train_loss += loss.item()
            epoch_train_true = accuracy_fn(y_batch, y_pred)
            epoch_train_total += len(y_batch)
        
        # update train accuracy & loss statistics
        epoch_train_accuracy = (epoch_train_true/epoch_train_total) * 100
        epoch_train_loss /= (len(train_dl.dataset)/train_dl.batch_size)

        # disable gradient calculation
        with torch.no_grad():
            # enumerate mini batches
            for idx, (X_batch, y_batch) in enumerate(val_dl):
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                # compute the models output
                test_logits = mlp(X_batch)
                test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
                # calculate loss
                loss = loss_fn(test_logits, y_batch)

                # update validation counters
                epoch_validation_loss += loss.item()
                epoch_validation_true = accuracy_fn(y_batch, test_pred)
                epoch_validation_total += len(y_batch)
        
        # update validation accuracy & loss statistics
        epoch_validation_accuracy = (epoch_validation_true/epoch_validation_total) * 100
        epoch_validation_loss /= (len(val_dl.dataset)/val_dl.batch_size)

        # update global epochs statistics
        loss_train.append(epoch_train_loss)
        accuracy_train.append(epoch_train_accuracy)
        loss_validation.append(epoch_validation_loss)
        accuracy_validation.append(epoch_validation_accuracy)

        # log metrics to WandB
        wandb.log({"train loss": epoch_train_loss, 
                  "val_loss": epoch_validation_loss,
                  "train accuracy": epoch_train_accuracy,
                  "validation accuracy": epoch_validation_accuracy})

        # print
        if epoch == (n_epochs - 1): # 99:
            print(
                f'Epoch {epoch}/{n_epochs}: \
                train loss {loss_train[-1]}, \
                validation loss {loss_validation[-1]}, \
                train accuracy {accuracy_train[-1]}, \
                validation accuracy {accuracy_validation[-1]}'
            )

In [44]:
def evaluation(mlp, test_dl):
    y_pred_all, y_test_all = list(), list()

    total = 0
    correct = 0

    with torch.no_grad():
        for X_batch, y_batch in test_dl:
            # outputs = mlp(X_batch)
            # evaluate on test set
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_hat = mlp(X_batch)
            test_pred = torch.softmax(y_hat, dim=1).argmax(dim=1)
            epoch_validation_true = accuracy_fn(y_batch, test_pred)
            
            print('Accuracy epoch: %.3f' % ((epoch_validation_true / len(y_batch)) * 100))
            y_pred_all.extend(test_pred.cpu().numpy())
            y_test_all.extend(y_batch.cpu().numpy())
            #total += len(y_batch)
            correct += epoch_validation_true

    # return y_test_all, y_pred_all
    acc = ((correct / len(test_dl.dataset)) * 100)
    print('Accuracy total: %.3f' % acc)
    wandb.log({'test accuracy': acc})

    report = classification_report(y_test_all, y_pred_all, target_names=['0', '1', '2', '3'], output_dict=True)
    #print(report['precision'])
    wandb.log({"macro avg precision": report['macro avg']['precision'], 
                "macro avg recall": report['macro avg']['recall'],
                "macro avg f1-score": report['macro avg']['f1-score'],
                "macro avg support": report['macro avg']['support']})
    
    wandb.log({"weighted avg precision": report['weighted avg']['precision'], 
                "weighted avg recall": report['weighted avg']['recall'],
                "weighted avg f1-score": report['weighted avg']['f1-score'],
                "weighted avg support": report['weighted avg']['support']})
    return report

In [45]:
def wandb_sweep(): #-> Tuple[List, List, List, List]:
    """ 
    Train the model
    n_epochs, mlp, optimizer, loss_fn, 
              train_dl, val_dl, device, default_config
    """    
    # start a new wandb run to track this script
    wandb.init(
        # set the wandb project where this run will be logged
        project="NSIETE_zad1_final",
        # track hyperparameters and run metadata
        config=default_config
    )

    config = wandb.config

    learning_rate = config.learning_rate    #  # 0.001 
    input_size = 20                         # number of features
    # hidden_size = config.hidden_size        #  #64         # number of features in hidden state
    num_classes = 4                 # number of output classes 

    n_epochs = config.epochs        
    batch_size = config.batch_size  

    train_dl = DataLoader(tf_train, batch_size=batch_size, shuffle=True, num_workers=0)
    val_dl = DataLoader(tf_val, batch_size=batch_size, shuffle=False, num_workers=0)
    test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=0) #.astype('float32')

    mlp = MLP( 
        n_inputs=input_size, 
        n_classes=num_classes,
        lr=learning_rate,
        config=config)
    mlp.to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(mlp.parameters(), lr=learning_rate)
    train_mlp(n_epochs, mlp, optimizer, loss_fn, 
              train_dl, val_dl, device, batch_size)
    
    classifiaction_report = evaluation(mlp, test_dl)
    pprint(classifiaction_report)
    # return loss_train, accuracy_train, loss_validation, accuracy_validation

In [48]:
wandb.agent(sweep_id, function=wandb_sweep, count=10)

wandb: Agent Starting Run: 8zwcbboa with config:
wandb: 	batch_size: 256
wandb: 	dense1: 256
wandb: 	epochs: 100
wandb: 	learning_rate: 0.043535698978630466
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch 99/100:                 train loss 1.5183715066792052,                 validation loss 1.9561311509874133,                 train accuracy 1.6049382716049383,                 validation accuracy 23.88888888888889
Accuracy epoch: 21.000
Accuracy total: 21.000
{'0': {'f1-score': 0.06153846153846154,
       'precision': 1.0,
       'recall': 0.031746031746031744,
       'support': 63},
 '1': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 49},
 '2': {'f1-score': 0.03278688524590164,
       'precision': 0.05263157894736842,
       'recall': 0.023809523809523808,
       'support': 42},
 '3': {'f1-score': 0.3466666666666666,
       'precision': 0.21787709497206703,
       'recall': 0.8478260869565217,
       'support': 46},
 'accuracy': 0.21,
 'macro avg': {'f1-score': 0.11024800336275745,
               'precision': 0.31762716847985883,
               'recall': 0.22584541062801933,
               'support': 200},
 'weighted avg': {'f1-score': 0.10600319461958806,
        

macro avg f1-score,▁
macro avg precision,▁
macro avg recall,▁
macro avg support,▁
test accuracy,▁
train accuracy,▄▅▆▁▄▂▆▃▅▄▆▄▄▄▇▆▅▅▃▃▄▃▃▅▄▅▅▃▆▆▆▄█▄▄▅▇▃▄▇
train loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation accuracy,▂▅█▆▄▃▅▄▄▆▅▃▅▂▇▇▄▅▃▃▅▆▇▄▅▂▂▅▃▆▂▅▃▆▇▅▁▆▄▄
weighted avg f1-score,▁
weighted avg precision,▁


wandb: Agent Starting Run: haee616o with config:
wandb: 	batch_size: 1024
wandb: 	dense1: 512
wandb: 	epochs: 500
wandb: 	learning_rate: 0.006145288920205471
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch 499/500:                 train loss 0.514247809516059,                 validation loss 3.9795742458767362,                 train accuracy 30.925925925925924,                 validation accuracy 72.77777777777777
Accuracy epoch: 73.000
Accuracy total: 73.000
{'0': {'f1-score': 0.8750000000000001,
       'precision': 1.0,
       'recall': 0.7777777777777778,
       'support': 63},
 '1': {'f1-score': 0.6236559139784946,
       'precision': 0.6590909090909091,
       'recall': 0.5918367346938775,
       'support': 49},
 '2': {'f1-score': 0.574468085106383,
       'precision': 0.5192307692307693,
       'recall': 0.6428571428571429,
       'support': 42},
 '3': {'f1-score': 0.8118811881188119,
       'precision': 0.7454545454545455,
       'recall': 0.8913043478260869,
       'support': 46},
 'accuracy': 0.73,
 'macro avg': {'f1-score': 0.7212512968009224,
               'precision': 0.730944055944056,
               'recall': 0.7259440007887212,
               'support': 200},
 'weig

...

Epoch 499/500:                 train loss 0.42022670227804304,                 validation loss 0.6600441826714409,                 train accuracy 4.444444444444445,                 validation accuracy 79.44444444444444
Accuracy epoch: 81.000
Accuracy total: 81.000
{'0': {'f1-score': 0.9344262295081968,
       'precision': 0.9661016949152542,
       'recall': 0.9047619047619048,
       'support': 63},
 '1': {'f1-score': 0.7391304347826086,
       'precision': 0.7906976744186046,
       'recall': 0.6938775510204082,
       'support': 49},
 '2': {'f1-score': 0.6590909090909092,
       'precision': 0.6304347826086957,
       'recall': 0.6904761904761905,
       'support': 42},
 '3': {'f1-score': 0.8571428571428572,
       'precision': 0.8076923076923077,
       'recall': 0.9130434782608695,
       'support': 46},
 'accuracy': 0.81,
 'macro avg': {'f1-score': 0.797447607631143,
               'precision': 0.7987316149087156,
               'recall': 0.8005397811298433,
               'suppo

macro avg f1-score,▁
macro avg precision,▁
macro avg recall,▁
macro avg support,▁
test accuracy,▁
train accuracy,▂▁▁▃▂▃▅▅▃▄▄▆▅▅▃▅▅▇▆▆▇▆▇▄▃▆▆▆▇▆▇▅▇▇▇▇▇▇██
train loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation accuracy,▁▃▃▃▃▅▅▄▅▅▅▅▆▆▆▇▆▇▇▇▇▅▇▅▇▇▇▆▇█▇▇▇▇█▇█▇▇█
weighted avg f1-score,▁
weighted avg precision,▁


wandb: Agent Starting Run: qiteiskr with config:
wandb: 	batch_size: 512
wandb: 	dense1: 512
wandb: 	epochs: 1000
wandb: 	learning_rate: 0.03159096579743968
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch 999/1000:                 train loss 1.751537181712963,                 validation loss 3.9504075792100695,                 train accuracy 1.3580246913580247,                 validation accuracy 20.555555555555554
Accuracy epoch: 23.000
Accuracy total: 23.000
{'0': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 63},
 '1': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 49},
 '2': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 42},
 '3': {'f1-score': 0.3739837398373984,
       'precision': 0.23,
       'recall': 1.0,
       'support': 46},
 'accuracy': 0.23,
 'macro avg': {'f1-score': 0.0934959349593496,
               'precision': 0.0575,
               'recall': 0.25,
               'support': 200},
 'weighted avg': {'f1-score': 0.08601626016260162,
                  'precision': 0.0529,
                  'recall': 0.23,
                  'support': 200}}


macro avg f1-score,▁
macro avg precision,▁
macro avg recall,▁
macro avg support,▁
test accuracy,▁
train accuracy,▅█▇█▆▃▂▆▄▃▂▂▃▃▄▃▁▄▃▂▄▂▃▅▃▃▂▃▄▂▅▃▃▄▂▃▃▃▄▄
train loss,▆▂▁▂▂██▆▆███████████████████████████████
val_loss,█▁▁▂▂▄▄▃▆▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
validation accuracy,▅█▇█▇▂▁▇▂▃▂▃▂▂▃▂▂▂▁▁▁▃▁▃▃▁▃▁▁▃▁▁▁▁▁▃▃▃▁▁
weighted avg f1-score,▁
weighted avg precision,▁


In [49]:
wandb.finish()

In [50]:
api = wandb.Api()
sweep = api.sweep("xmatovice/uncategorized/sweeps/" + sweep_id)

best_run = sweep.best_run()
print(best_run.id)
pprint(best_run.config)

wandb: Sorting runs by -summary_metrics.validation_accuracy


2hfg3dq6
{'batch_size': 128,
 'dense1': 256,
 'epochs': 100,
 'learning_rate': 0.03648030109469176}
